# <span style="color:green"> Import libraries<span style="color:green">

In [1]:
#Import all the required libraries
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn 
import random
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_regression
from sklearn.linear_model import Lasso, LassoCV, Ridge, RidgeCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error, explained_variance_score
from sklearn.feature_selection import f_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
import pybiomart
from biomart import BiomartServer
from sklearn.feature_selection import RFECV
import xgboost as xgb

# <span style="color:green"> ML for Feature Selection and Association <span style="color:green">

In [2]:
#Read the file 
df= pd.read_table("/home/Vaishnavi/Imputeted_data/Imputed_maf_0.01/GWAStutorialldlraw.raw", delim_whitespace= True)
print(df.head())
print("Shape of file", df.shape)

     FID  IID  PAT  MAT  SEX  PHENOTYPE  rs2436029:269101:C:T_T  \
0  10004    1    0    0    2         75                       2   
1  10005    1    0    0    1         69                       2   
2  10007    1    0    0    1        108                       2   
3  10008    1    0    0    1         94                       2   
4  10009    1    0    0    1         92                       2   

   rs117942522:320955:C:A_A  rs17158845_T  rs4881331:517451:C:G_G  ...  \
0                         0             2                       2  ...   
1                         0             2                       2  ...   
2                         0             2                       2  ...   
3                         0             2                       2  ...   
4                         0             2                       2  ...   

   rs4880117_A  rs11137286_T  rs11137126_G  rs9410127_C  rs3125786_T  \
0            2             2             2            2            2   
1       

In [3]:
new_col_names= list(df.columns[:6])
new_col_names.extend([col_name[:-2] for col_name in df.columns[6:]])
df.columns=new_col_names
df.columns

Index(['FID', 'IID', 'PAT', 'MAT', 'SEX', 'PHENOTYPE', 'rs2436029:269101:C:T',
       'rs117942522:320955:C:A', 'rs17158845', 'rs4881331:517451:C:G',
       ...
       'rs4880117', 'rs11137286', 'rs11137126', 'rs9410127', 'rs3125786',
       'rs11137198', 'rs7865267', 'rs1378954:141005656:G:A', 'rs9777369',
       '9:141049179:C:T'],
      dtype='object', length=38836)

In [4]:
#X is Independent feature and y is dependent feature
X= df.iloc[:, 6:]
print("Shape of X", X.shape)
y= df.PHENOTYPE
print("Shape of y", y.shape)

Shape of X (804, 38830)
Shape of y (804,)


## R-squared function

In [5]:
def r2_score_manual(y_true, y_pred):
    # Convert inputs to NumPy arrays
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)
    
    # Calculate the mean of the actual values
    y_mean = np.mean(y_true)
    
    # Residual sum of squares
    ss_res = np.sum((y_true - y_pred) ** 2)
    
    # Total sum of squares
    ss_tot = np.sum((y_true - y_mean) ** 2)
    
    # R² score
    r2 = 1 - (ss_res / ss_tot)
    return r2

## <span style="color:green"> LASSO <span style="color:green">

In [6]:
# Create and fit the LASSOCV model select 10,000 SNP
lasso_cv = linear_model.LassoCV(alphas= [0.000065], cv=5, random_state=2000)

# Fit the model to the data
lasso_cv.fit(X,y)

#Print the optimal alpha value
print ("Optimal alpha:", lasso_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
lasso_snps_ldl = np.where(lasso_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(lasso_snps_ldl), "selected SNPs")

#Final SNPs
l1= X.iloc[:,lasso_snps_ldl]
print("print shape of selected snps by lasso",l1.shape)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.538e+02, tolerance: 8.481e+01
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.497e+02, tolerance: 8.188e+01
  model = cd_fast.enet_coordinate_descent(
/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Du

Optimal alpha: 6.5e-05
5055 selected SNPs
print shape of selected snps by lasso (804, 5055)


/apps/compiler/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.909e+02, tolerance: 9.977e+01
  model = cd_fast.enet_coordinate_descent(


### Linear regression

In [7]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(l1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(l1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = l1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#lasso_LR_sigSNP=l1.iloc[:, selected_p_indices]
#lasso_LR_sigSNP
#lasso_LR_sig_col= list(lasso_LR_sigSNP.columns)
#lasso_LR_sig_col

Y.shape (804,)
x_train=(562, 5055), x_test=(242, 5055), Y_train= (562,), Y_test= (242,)
r2 is  0.21330028770830345
r2 is  0.21330028770830345
mean_absolute_error 24.622159361950125
mean_squared_error  1226.6967750084948
explained_variance_score 0.21415028471685826


### Random Forest

In [8]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  0.02023897605534941
r2 is  0.02023897605534941
mean_absolute_error 28.38908758137469
mean_squared_error  1527.7362754472294
explained_variance_score 0.02061580031097543


### Support Vector Machine

In [9]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  0.24803414719518657
r2 is  0.24803414719518657
mean_absolute_error 24.248212205770606
mean_squared_error  1172.5364483292863
explained_variance_score 0.2517321677242027


### XGBoost

In [10]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.0071639919736514734
r2 is  -0.0071639919736514734
mean_absolute_error 28.69527681208839
mean_squared_error  1570.4655811551386
explained_variance_score -0.007063029289272826


## <span style="color:green"> Ridge<span style="color:green">

In [11]:
# Create and fit the RidgeCV model select 5000 SNP
ridge_cv = linear_model.RidgeCV(alphas=[0.005], cv=5, scoring='neg_mean_squared_error')
ridge_cv.fit(X, y)

# Optimal alpha value
print("Optimal alpha:", ridge_cv.alpha_)

# Indices of the SNPs with non-zero coefficients
ridge_snps_ldl = np.where(ridge_cv.coef_)[0]

# Select coefficients with non-zero values
selected_features = ridge_cv.coef_ != 0

# Select coefficients of selected SNPs
coefficients = ridge_cv.coef_[selected_features]

# Get the indices of the top 5000 SNPs based on descending coefficients
top_n = 5000
sorted_indices = np.argsort(-coefficients)
selected_indices = sorted_indices[:top_n]

# Print the indices of selected SNPs and the number of selected SNPs
print("Selected SNPs indices based on top coefficients:", selected_indices)
print(len(selected_indices), "selected SNPs")

#Maximum Coefficient
print("Maximum value of coefficient", coefficients.max())

#Final SNPs
R1= X.iloc[:,selected_indices]
print("print shape of selected snps by Ridge",R1.shape)


Optimal alpha: 0.005
Selected SNPs indices based on top coefficients: [24344 11183 24683 ... 30323 11320 14165]
5000 selected SNPs
Maximum value of coefficient 0.43818829476400195
print shape of selected snps by Ridge (804, 5000)


### Linear Regression

In [12]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(R1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (804,)
x_train=(562, 5000), x_test=(242, 5000), Y_train= (562,), Y_test= (242,)


In [13]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(R1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = R1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#Ridge_LR_sigSNP=R1.iloc[:, selected_p_indices]
#Ridge_LR_sigSNP
#Ridge_LR_sig_col= list(Ridge_LR_sigSNP.columns)
#Ridge_LR_sig_col

r2 is  0.7267563043767364
r2 is  0.7267563043767364
mean_absolute_error 14.901300123141873
mean_squared_error  426.0674752708918
explained_variance_score 0.7267570487324846


### Random Forest

In [14]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  0.005483246850291668
r2 is  0.005483246850291668
mean_absolute_error 29.072424524803633
mean_squared_error  1550.744807350736
explained_variance_score 0.009253280654187068


### Support Vector Machine

In [15]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  0.7581709976637709
r2 is  0.7581709976637709
mean_absolute_error 13.916722121066252
mean_squared_error  377.0827072063047
explained_variance_score 0.7613266921803723


### XGBoost

In [16]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score -0.09379164806354656
r2 is  -0.09379164806354656
mean_absolute_error 30.172190737133185
mean_squared_error  1705.5436353245773
explained_variance_score -0.09289767366345614


## <span style="color:green"> Elastic net<span style="color:green">

In [17]:
# Create and fit the Elasticnet model
elastic_cv = linear_model.ElasticNetCV(l1_ratio=[0.5],alphas=[0.136], cv=5, random_state= 2000)
elastic_cv.fit(X, y)

# Print the optimal alpha value
print ("Optimal alpha:", elastic_cv.alpha_)
print("Optimal l1_ratio: ", elastic_cv.l1_ratio_)

# Indices of the SNPs with non-zero coefficients
elastic_snps_ldl = np.where(elastic_cv.coef_)[0]

# See how many SNPs have a non-zero coefficient
print(len(elastic_snps_ldl), "selected SNPs")

#Final SNPs
E1= X.iloc[:,elastic_snps_ldl]
print("print shape of selected snps by Elastic net",E1.shape)

Optimal alpha: 0.136
Optimal l1_ratio:  0.5
5000 selected SNPs
print shape of selected snps by Elastic net (804, 5000)


### Linear Regression

In [18]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(E1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (804,)
x_train=(562, 5000), x_test=(242, 5000), Y_train= (562,), Y_test= (242,)


In [19]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(E1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = E1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#Elasticnet_LR_sigSNP=E1.iloc[:, selected_p_indices]
#Elasticnet_LR_sigSNP
#Elasticnet_LR_sig_col= list(Elasticnet_LR_sigSNP.columns)
#Elasticnet_LR_sig_col

r2 is  0.8355327633794449
r2 is  0.8355327633794449
mean_absolute_error 10.754641248473234
mean_squared_error  256.45290776741444
explained_variance_score 0.8355334697053634


### Random Forest

In [20]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  0.06194979196028527
r2 is  0.06194979196028527
mean_absolute_error 28.26400428155401
mean_squared_error  1462.6968168658768
explained_variance_score 0.06239113068281987


### Support Vector Regression

In [21]:
from sklearn.inspection import permutation_importance

svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale', cache_size=800)
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  0.8897982986786398
r2 is  0.8897982986786398
mean_absolute_error 8.658024483347209
mean_squared_error  171.8369404477901
explained_variance_score 0.8897983479276816


### XGBoost for Elastic-net

In [22]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 score 0.05744118343521365
r2 is  0.05744118343521365
mean_absolute_error 28.056912634983536
mean_squared_error  1469.7270667198777
explained_variance_score 0.05781458120083338


## <span style="color:green"> Mutual information <span style="color:green">

In [23]:
# Create and fit the Mutual_info model
mutual_info= mutual_info_regression(X, y, random_state=2000)
print("Length of mutal info", len(mutual_info))
mutual_info= pd.Series(mutual_info, index=df.columns[6:])

#Sort mutual_info values descending order
vdesc= mutual_info.sort_values(ascending=False)
vdesc_nonzero= vdesc[vdesc!=0]
print("Maximum value of mutualinfo", vdesc_nonzero.max)
print("Length of mutal info non zero", len(vdesc_nonzero))
k= np.where(vdesc_nonzero)
first_col= k[0]

#Selected 5000 SNPs
top5000 = 5000
mi_pos= first_col[:top5000]

#Final SNPs
M1= X.iloc[:,mi_pos]
print("print shape of selected snps by Mutual_info",M1.shape)

Length of mutal info 38830
Maximum value of mutualinfo <bound method NDFrame._add_numeric_operations.<locals>.max of rs9881090                  9.749381e-02
rs12649176                 9.574809e-02
rs16824778                 9.354285e-02
rs17681524                 9.232919e-02
rs1472491                  9.055629e-02
                               ...     
rs17029090                 2.040916e-06
rs56002751:39625366:A:C    8.888933e-07
rs12555893                 5.370780e-07
rs966078                   5.363469e-07
rs7523161                  2.856900e-07
Length: 19050, dtype: float64>
Length of mutal info non zero 19050
print shape of selected snps by Mutual_info (804, 5000)


### Linear Regression

In [24]:
Y= df.PHENOTYPE.values
print("Y.shape",Y.shape)
x_train, x_test, Y_train, Y_test= train_test_split(M1, Y, test_size=0.3, random_state=2000)
print(f"x_train={x_train.shape}, x_test={x_test.shape}, Y_train= {Y_train.shape}, Y_test= {Y_test.shape}")
scaler= StandardScaler()
x_train= scaler.fit_transform(x_train)
x_test= scaler.transform(x_test)

Y.shape (804,)
x_train=(562, 5000), x_test=(242, 5000), Y_train= (562,), Y_test= (242,)


In [25]:
random.seed(2000)
model = linear_model.LinearRegression()
model.fit(x_train, Y_train)
y_pred = model.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

#f_statistic, p_value = f_regression(M1, Y)
#selected_p_indices = np.where(p_value < 5e-04)[0]
#selected_p_values = p_value[selected_p_indices]
#print("Selected p-values:")
#print(selected_p_values)
#print("Number of selected features:", len(selected_p_indices))
#selected_f_values = f_statistic[selected_p_indices]
#print("Selected F-values:")
#print(selected_f_values)
#top_f_indices = np.argsort(-selected_f_values)
#print("Indices of selected features with top F-values:")
#print(top_f_indices)
#top_f_features = M1.iloc[:, selected_p_indices[top_f_indices]]
#print("Top selected features:")
#print(top_f_features)
#MI_LR_sigSNP=M1.iloc[:, selected_p_indices]
#MI_LR_sigSNP
#MI_LR_sig_col= list(MI_LR_sigSNP.columns)
#MI_LR_sig_col

r2 is  -0.06454135288435103
r2 is  -0.06454135288435103
mean_absolute_error 29.355031468962512
mean_squared_error  1659.9338019869726
explained_variance_score -0.06444087675509103


### Random Forest

In [28]:
# Create a Random Forest Regressor
# Create a Random Forest Regressor
rf_regressor = RandomForestRegressor(n_estimators=100, max_depth=9, min_samples_leaf=4, min_samples_split=2, 
                                     n_jobs=70,random_state=2000)

rf_regressor.fit(x_train, Y_train)


# Make predictions on the test set using the model with selected features
y_pred= rf_regressor.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))

r2 is  -6.896174307291503e-05
r2 is  -6.896174307291503e-05
mean_absolute_error 29.06893318393401
mean_squared_error  1559.402337370436
explained_variance_score -3.932338212231379e-05


### Support Vector Regressor

In [26]:
svr = SVR(kernel = 'sigmoid' , C=100, epsilon= 0.01, gamma= 'scale')
svr.fit(x_train, Y_train)
y_pred= svr.predict(x_test)

score= r2_score(Y_test, y_pred)
print("r2 is ", score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 is  -0.12759580712612117
r2 is  -0.12759580712612117
mean_absolute_error 30.620929464937387
mean_squared_error  1758.2542849613203
explained_variance_score -0.1259701588287958


### XGBoost 

In [27]:
regressor=xgb.XGBRegressor(learning_rate= 0.01, n_estimators=900,  max_depth=10, eval_metric='rmsle', random_state=2000)
regressor.fit(x_train, Y_train)
y_pred= regressor.predict(x_test)
score= r2_score(Y_test, y_pred)
print("r2 score",score)
r2_man = r2_score_manual(Y_test, y_pred)
print("r2 is ",r2_man)
h= mean_absolute_error(Y_test, y_pred)
print("mean_absolute_error", h)
print("mean_squared_error ", mean_squared_error(Y_test, y_pred))
print("explained_variance_score", explained_variance_score(Y_test, y_pred))


r2 score -0.07985364874815382
r2 is  -0.07985364874815382
mean_absolute_error 30.78732646595348
mean_squared_error  1683.8101853904764
explained_variance_score -0.07982777063303526
